In [14]:
!pip install pandas
!pip install nltk
!pip install scikit-learn
!pip install scikit-multilearn

In [5]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.metrics import classification_report, hamming_loss, accuracy_score
import warnings

warnings.filterwarnings('ignore')

import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /home/glkaiky/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/glkaiky/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/glkaiky/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/glkaiky/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
try:
    df = pd.read_csv('data/train.csv')
except FileNotFoundError:
    print("Erro: Arquivo 'train.csv' não encontrado. Faça o download do Kaggle e coloque na mesma pasta do notebook.")
    # Cria um dataframe de exemplo para o código não quebrar
    df = pd.DataFrame({
        'comment_text': ['This is a good, clean comment.', 'This is an obscene, hateful comment.', 'This is an insult.'],
        'toxic': [0, 1, 1], 'severe_toxic': [0, 0, 0], 'obscene': [0, 1, 0],
        'threat': [0, 0, 0], 'insult': [0, 0, 1], 'identity_hate': [0, 1, 0]
    })

In [7]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    # Converte para minúsculas
    text = text.lower()
    # Remove pontuação e caracteres não-alfabéticos
    text = re.sub(r'[^a-z\s]', '', text)
    # Tokeniza
    tokens = word_tokenize(text)
    # Remove stop words e aplica lematização
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return " ".join(tokens)

print("Iniciando pré-processamento de texto...")
# Aplicando a limpeza em uma amostra para agilizar (remova .sample() para rodar em tudo)
# df['clean_comment'] = df['comment_text'].sample(n=10000, random_state=42).apply(clean_text)
# df = df.dropna(subset=['clean_comment'])
df['clean_comment'] = df['comment_text'].apply(clean_text)
print("Pré-processamento concluído.")


Iniciando pré-processamento de texto...
Pré-processamento concluído.


In [8]:
X = df['clean_comment']
y = df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]

In [9]:
print("Iniciando vetorização com TF-IDF...")
vectorizer = TfidfVectorizer(max_features=5000) # Usando as 5000 features mais relevantes
X_tfidf = vectorizer.fit_transform(X)
print("Vetorização concluída.")


Iniciando vetorização com TF-IDF...
Vetorização concluída.


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)


In [11]:
print("Iniciando treinamento do modelo...")
base_classifier = LogisticRegression(solver='liblinear')
classifier = BinaryRelevance(classifier=base_classifier, require_dense=[False, True])
classifier.fit(X_train, y_train)
print("Treinamento concluído.")


Iniciando treinamento do modelo...
Treinamento concluído.


In [12]:
print("Fazendo previsões no conjunto de teste...")
predictions = classifier.predict(X_test)

Fazendo previsões no conjunto de teste...


In [13]:
loss = hamming_loss(y_test, predictions)
subset_accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions, target_names=y.columns)

In [16]:
print("\n--- Resultados da Avaliação ---")
print(f"Hamming Loss: {loss:.8f}")
print(f"Acurácia (Subset Accuracy): {subset_accuracy:.8f}") # Imprime a nova métrica
print("\nRelatório de Classificação:")
print(report)


--- Resultados da Avaliação ---
Hamming Loss: 0.01907149
Acurácia (Subset Accuracy): 0.91988093

Relatório de Classificação:
               precision    recall  f1-score   support

        toxic       0.90      0.62      0.74      3056
 severe_toxic       0.58      0.26      0.36       321
      obscene       0.91      0.64      0.75      1715
       threat       0.71      0.14      0.23        74
       insult       0.83      0.53      0.65      1614
identity_hate       0.73      0.16      0.26       294

    micro avg       0.87      0.57      0.69      7074
    macro avg       0.78      0.39      0.50      7074
 weighted avg       0.86      0.57      0.68      7074
  samples avg       0.06      0.05      0.05      7074

